![](https://github.com/GabrielTrentino/Projetos-de-Cursos/blob/master/03-DSNP/00%20-%20Imagens/05%20-%20ImovelSaoPaulo.png?raw=true)
# Preço de Venda e Aluguel para Imóveis em São Paulo:

O Data set do Preço de Venda e Aluguel para Imóveis em São Paulo está disponibilizado pelo [link do kaggle](https://www.kaggle.com/argonalyst/sao-paulo-real-estate-sale-rent-april-2019) possuindo mais de 13000 entradas sobre imóveis em São Paulo. Essas entradas são divididas entre Venda e Aluguel, e será utilizado somente a parte para as vendas nesse notebook. Lembrando que esse Data set é uma parcela do que foi realmente feito no [notebook de competição](https://github.com/GabrielTrentino/Competicoes/blob/master/Pre%C3%A7o_imoveis_S%C3%A3o_Paulo.ipynb).

Os codigos escritos nesse notebook tem como a finalidade de mostrar os processos necessários para o Deploy em uma aplicação web. Alguns processos como análise exploratória ou análise implicita não estarão disponíveis nesse atual notebook. Esse notebook teve como inspiração as aulas do curso de Data Science na Prática.

A aplicação Web teve esse resultado:

![](https://github.com/GabrielTrentino/Projetos-de-Cursos/blob/master/03-DSNP/00%20-%20Imagens/05-TelaWebAppSP.png?raw=true)

#Primeiros Passos:

Nessa primeira parte do notebook, é instalado as funções e bibliotecas primeiramente necessárias (podendo ser instalada outras ao decorrer do estudo), para que em seguida, possamos utilizar o arquivo `csv` disponibilizado no Kaggle. Esse arquivo é convertido para o tipo Data Frame do pandas e, em seguida, é mostrado as primeiras 5 entradas com a função "`.head()`".

##Importando as Bibliotecas:

Inicialmente será importado as bibliotecas: pandas e numpy para manipulação dos dados e, em seguida, serão instaladas outras bibliotecas para a elaboração do modelo.

In [ ]:
# Importar os pacotes necessários
import pandas as pd
import numpy as np

## Importando o Data Set:

In [ ]:
# Importar o dataset para um dataframe
df = pd.read_csv('sao-paulo-properties-april-2019.csv')

# Ver as 5 primeiras entradas
df.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


##Dicionário das variáveis:

Nessa parte do notebook é descrito quais são as variáveis presentes no Data Set e o que elas significam. É uma parte crucial para entendermos o que estaremos requisitando do modelo e do usuário durante a aplicação Web.

* `Price` - Preço Final anunciado em Reais (R$).
* `Condo` - Taxa de condomínio (valores desconhecidos = 0).
* `Size` - Área da Propriedade em m² (apenas áreas privadas).
* `Rooms` - Número de quartos.
* `Toilets` - Número de banheiros (todos os banheiros, inclue suíte).
* `Suites` - Numero de quartos com banheiro exclusivo.
* `Parking` - Quantidade de vagas de garagem.
* `Elevator`- Valor Binário: 1 indica que há elevador, 0 não há.
* `Furnished` - Valor Binário: 1 indica que a propriedade é mobilhado, 0 não é mobilhado.
* `Swimming Pool`- Valor Binário: 1 indica que há piscina, 0 não há.
* `New` - Valor Binário: 1 indica que a propriedade foi construida recentemente, 0 não é recente.
* `District` - Setor/Cidade em que a propriedade está localizada.
* `Negotiation Type` - O imóvel está para Venda (`Sale`) ou Aluguel (`Rent`)?
* `Property Type` - Tipo de Imóvel, o Dataset é focado para apartamentos, apenas.
* `Latitude` - Latitude do imóvel.
* `Longitude` - Longitude do imóvel.

A variável `Property Type` não será util para o nosso estudo, pois estaremos trabalhando apenas com apartamentos. E so iremos trabalhar com negociações do tipo Venda (`Sale`) da variável `Negotiation Type`. Essa escolha se deu devido à maior precisão desse modelo durante fases de validação no [notebook de competição](https://github.com/GabrielTrentino/Competicoes/blob/master/Pre%C3%A7o_imoveis_S%C3%A3o_Paulo.ipynb), citado anteriormente.

##Limpeza de Outliers:

Os outliers a serem removidos serão aqueles que estarão a 3 desvio padrões da média dos dados.

In [ ]:
# Criando função para tirar variáveis de uma lista:
def tirar_outlier(df, lista):
  df_clean = df[lista]
  for col, dados in df_clean.iteritems():
    df_clean = df_clean[((df_clean[col] - df_clean[col].median()) / df_clean[col].std()).abs() < 3]
    df_clean = df_clean[np.abs(df_clean[col]-df_clean[col].median()) <= (3*df_clean[col].std())]

  valores_retirados = df.shape[0] - df_clean.shape[0]
  print('Foi retirado {} valores de {} ({:.2f})%'.format(valores_retirados, df.shape[0], valores_retirados/df.shape[0]*100))
  print('Ficamos com {} resultados'.format(df_clean.shape[0]))

  for i in range(0, len(df)):
    if i not in df_clean.index:
        df = df[df.index != i]

  df = df.reset_index(drop=True)
  return df

# Retirando o Outlier de Latitude e Longitude:
df = tirar_outlier(df, ['Latitude', 'Longitude'])

Foi retirado 1032 valores de 13640 (7.57)%
Ficamos com 12608 resultados


## Separando os bairros da coluna `District`:

Ao utilizar "`.apply()`" retiramos as informações da cidade (São Paulo) e mantemos apenas o bairro.

In [ ]:
df['District'] = df['District'].apply(lambda x: x.split('/')[0])
df.head(2)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.550239,-46.480718


## Criando o Data Frame com os valores médios para os bairros:

Iremos criar um Data Frame com os valores médios de Latitude e Longitude para que possamos consultar pelo Pycharm quando o usuário selecionar o bairro desejado. Dessa forma, teremos valores de Latitude e Longitude mais próximos e precisos.

In [ ]:
# Criando Data Frame vazio:
Bairros_LatLong = pd.DataFrame()

# Agrupando os valores médios por bairro:
Bairros_LatLong['Latitude'] = df.Latitude.groupby(df.District).mean()
Bairros_LatLong['Longitude'] = df.Longitude.groupby(df.District).mean()

# Gerando um arquivo csv com os valores médios:
Bairros_LatLong.to_csv('Bairros_LatLong.csv')

# Visualizando o Data Frame:
Bairros_LatLong

,Latitude,Longitude
District,,
Alto de Pinheiros,-23.546400,-46.711485
Anhanguera,-23.591509,-46.672088
Aricanduva,-23.553916,-46.522669
Artur Alvim,-23.544731,-46.486363
Barra Funda,-23.523698,-46.662652
...,...,...
Vila Matilde,-23.536455,-46.526908
Vila Olimpia,-23.597109,-46.682245
Vila Prudente,-23.588982,-46.575585


## Retirando Coluna de `District` e `Property Type`:

In [ ]:
# Retirando as colunas de District e Property Type:
df.drop(['District', 'Property Type'], axis = 1, inplace = True)

##Criando o Data Frame para Sale:

Aqui é utilizado o Data Frame `df` com a finalidade de elaborar um novo Data Frame contendo apenas os dados de imóveis que estão disponíveis para venda. O Data Frame `Sale` é escolhido sobre o `Rent` devido à uma precisão melhor quando comparadas.

In [ ]:
# Criando o Data Frame com os dados de venda
Sale = df[df['Negotiation Type'] == 'sale'].drop('Negotiation Type', axis = 1)
Sale.head(1)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude
4504,732600,1000,74,1,2,1,2,1,0,1,0,-23.552129,-46.692244


In [ ]:
# Retirando os Outliers do Data Frame:
print('Para o Data Frame Sale:')
Sale_clean = tirar_outlier(Sale, ['Latitude', 'Longitude', 'Price', 'Rooms', 'Toilets', 'Suites', 'Parking'])

Para o Data Frame Sale:
Foi retirado 787 valores de 5917 (13.30)%
Ficamos com 5130 resultados


#Modelo de Regressão Linear:

Para esse modelo de Deploy, iremos utilizar a Regressão Linear como modelo preditivo, uma vez que o XGBoost trouxe problemas técnicos na instalação no Pycharm.

In [ ]:
# Instalando as bibiotecas necessárias:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Criando as variáveis X e Y:
X = Sale_clean.drop(['Price'], axis = 1)
y = Sale_clean[['Price']]

# Separando os dados de treino e teste:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Elaborando o modelo, treinamento e validação:
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
resultados = r2_score(y_test, y_pred)

# Imprimindo o R²:
resultados.round(3)

0.725

#Exportando os Modelos:

Para utilizarmos os dados no Pycharm, devemos criar um arquivo contendo o modelo e os nomes das features para serem expotados para o Pycharm, para que assim possamos criar a aplicação Web com esses dados.

In [ ]:
# Importando o Pickle para exportar o modelo.
import pickle

# Criando o arquivo com o modelo de Machine Learning:
with open('modelo_simples.pkl', 'wb') as file:
    pickle.dump(modelo, file)

In [ ]:
# Extraindo os nomes das features:
features_simples = X_train.columns.values

# Criando o arquivo com os nomes das features:
with open('features_simples.names', 'wb') as file:
    pickle.dump(features_simples, file)

#Códigos no Pycharm:

![](https://github.com/GabrielTrentino/Projetos-de-Cursos/blob/master/03-DSNP/00%20-%20Imagens/05-TelaWebAppSP.png?raw=true)

Essa parte do Notebook, teremos uma simulação de como os dados são processados na linguagem Python, o [arquivo HTML](https://github.com/GabrielTrentino/Projetos-de-Cursos/blob/master/03-DSNP/05%20-%20Template%20-%20home.html) tem as informações do template, enquanto que o [WebApp](github.com/GabrielTrentino/Projetos-de-Cursos/blob/master/03-DSNP/05%20-%20Webapp.py) possui o código do que foi realizado no Pycharm.

In [ ]:
# Dados obtidos pela aplicação web:
user_inputs = {'District': 'Alto de Pinheiros', 'Condo': '55', 'Size': '33', 'Rooms': '1', 'Toilets': '1', 'Suites': '2', 'Parking': '0', 'Elevator': None, 'Furnished': None, 'Swimming Pool': '1', 'New':
 None}

features_names = features_simples

In [ ]:
# Criando um Data Frame vazio e colocando 0 para todas as colunas:
dados = pd.DataFrame(index=[0], columns=features_names)
dados = dados.fillna(value=0)

# Se os valores de user_inputs forem 'None', a variável recebera 0:
for i in user_inputs.items():
    if i[1] != None:
        dados[i[0]] = i[1]
    else:
        dados[i[0]] = 0

# Abrindo o arquivo das medias das Latitudes e Longitudes por bairro:
df_lat_long = pd.read_csv('Bairros_LatLong.csv')

# Inserido Latitude e Longitude de acordo com o Bairro selecionado:
dados.Latitude = df_lat_long[df_lat_long.District == user_inputs.get('District')].Latitude.values[0]
dados.Longitude = df_lat_long[df_lat_long.District == user_inputs.get('District')].Longitude.values[0]

# Retirando a variável 'District':
dados.drop('District', axis=1, inplace=True)

# Convertendo os dados para float:
dados = dados.astype(float)

# Prevendo o valor de venda do imóvel:
y_pred = modelo.predict(dados)[0][0]
y_pred

220027.42127412185